## Packages

In [1]:
import networkx as nx 
import pickle 
from networkx.algorithms.approximation import greedy_tsp # For approx TSP
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import math
import numpy as np
from matplotlib import pyplot as plt
from torch.utils.data import Dataset

## Helper functions

In [2]:

def tour_length(G, tour):
    """
    Compute the length of a tour. 
    """
    n = len(tour) - 1
    assert tour[0] == tour[-1], "Not valid tour"
    estimated = 0
    for i in range(n):
        estimated += G[tour[i]][tour[i + 1]]['weight']
    return estimated

def greedy_algorithm(G):
    """
    Run the value of the greedy approximation algorithm on graph G
    """
    return tour_length(G, greedy_tsp(G, weight='weight'))

def random_tour(G, seed = 42):
    """
    Return the value of a random tour
    """
    np.random.seed(seed)
    n = G.number_of_nodes()
    tour = [0]
    for i in range(1, n):
        next_node = np.random.choice([j for j in range(n) if j not in tour])
        tour.append(next_node)
    tour.append(0)
    return tour_length(G, tour)
    

def transformer_tsp(G, model, DEVICE = 'cpu'):
    """
    Evaluate your (trained) model on G
    """
    # Set the model in evaluation mode
    model.eval()

    # Note: number of edges is constant ed equal to n(n-1)/2
    n = G.number_of_nodes()
    E = G.number_of_edges()

    
    # Get node coordinates
    attr = nx.get_node_attributes(G, 'pos')
    x = []
    for i in range(n):
        x.append(torch.tensor(attr[i], dtype=torch.float32))

    # From list of tensors to tensor
    x = torch.stack(x)    

    tour = [0]
    y = torch.tensor(tour, dtype=torch.long)
    x = torch.stack(x)
    x = x.to(DEVICE).unsqueeze(0)
    y = y.to(DEVICE).unsqueeze(0)
    
    out = model(x, y)
    
    while len(tour) < n:
        _, idx = torch.topk(out, n, dim=2)
        for i in range(n):
            if idx[0,-1, i] not in tour:
                tour.append(idx[0, -1, i])
                break
        y = torch.tensor(tour)
        y = y.to(DEVICE).unsqueeze(0)
        out = model(x, y)
    
    tour = [int(i) for i in tour] + [0]
    return tour_length(G, tour)



def gap(G, model = None, model_GA = None, random_seed = 42, device = 'cpu'):
    """
    Compute the gap between the optimal solution on graph G and all the analyzed methods
    """
        
    # Optimal value (hard-coded in the graph)
    TSP = sum([G[i][j]['weight']*G[i][j]['tour'] for (i, j) in G.edges()]) # Optimal

    # Gaps dictionary
    gaps = {'greedy' : 0, 'random' : 0, 'transformer_tsp': 0, 'transformer_tsp_acc_grad': 0}
    gaps['greedy'] = 100* (greedy_algorithm(G) -  TSP) / TSP
    gaps['random'] = 100 * (random_tour(G, random_seed) - TSP) / TSP
    if model is not None:
        gaps['transformer_tsp'] = 100 * (transformer_tsp(G, model, DEVICE=device) - TSP) / TSP
    else:
        gaps['transformer_tsp'] = float('inf')
        
    if model_GA is not None:
        gaps['transformer_tsp_acc_grad'] = 100 * (transformer_tsp(G, model_GA, DEVICE=device) - TSP) / TSP
    else:
        gaps['transformer_tsp_acc_grad'] = float('inf')
    return gaps    
    

## Dataset & Dataloader

In [3]:
# Load the dummy dataset
with open('dummy_20_DLL_ass4.pkl', 'rb') as file:
 dummy_data = pickle.load(file)


single_item = dummy_data[0]
print(single_item)


print(f"Type of single_item: {type(single_item)}")

# Extract the graph from the tuple
graph = single_item[0]

# Get edge attributes
edge_attributes_weight = nx.get_edge_attributes(graph, 'weight')
edge_attributes_tour = nx.get_edge_attributes(graph, 'tour')


print("Edge attributes:")
print("- 'weight': ", edge_attributes_weight[next(iter(edge_attributes_weight))], "(Euclidean distance between the two nodes of the edge.)")

print("- 'tour': ", edge_attributes_tour[next(iter(edge_attributes_tour))],"(Binary value (0 or 1), indicating whether the edge is part of the optimal tour (1) or not (0).)")


# Get node attributes
node_attributes = nx.get_node_attributes(graph, 'pos')


print("Node attribute:")
print("- 'pos': ", node_attributes[0], "representing the 2D coordinates of the node.")

class TSPDataset(Dataset):
    def __init__(self, data, return_graph=False):
        self.data = data
        self.return_graph = return_graph

    def __len__(self): #Return the number of items in the dataset.
        return len(self.data)

    def __getitem__(self, idx):
        graph, tour = self.data[idx]
        if self.return_graph:
            return graph, tour  # Return the original NetworkX graph

        # Get the number of nodes in the graph
        n = graph.number_of_nodes()

        # Extract node coordinates (pos attribute)
        pos = nx.get_node_attributes(graph, 'pos')
        X = torch.tensor([[pos[i][0], pos[i][1]] for i in range(n)], dtype=torch.float32)

        # Ensure the tour starts and ends with node 0
        if tour[0] != 0:
            tour.insert(0, 0)
        if tour[-1] != 0:
            tour.append(0)

        # Convert the tour to a tensor
        y = torch.tensor(tour, dtype=torch.long)

        return X, y



# Load datasets from pickle files
with open('train_20_DLL_ass4.pkl', 'rb') as f:
    train_data = pickle.load(f)
with open('valid_20_DLL_ass4.pkl', 'rb') as f:
    val_data = pickle.load(f)
with open('test_20_DLL_ass4.pkl', 'rb') as f:
    test_data = pickle.load(f)

# Create Dataset objects using the TSPDataset class
train_dataset = TSPDataset(train_data)
val_dataset = TSPDataset(val_data)
test_dataset = TSPDataset(test_data)


BATCH_SIZE = 32

# Create DataLoaders for each dataset
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False) 



(<networkx.classes.graph.Graph object at 0x10b71cef0>, [0, 3, 14, 2, 9, 6, 19, 13, 12, 16, 7, 18, 8, 17, 5, 11, 10, 15, 1, 4, 0])
Type of single_item: <class 'tuple'>
Edge attributes:
- 'weight':  0.4287846201876535 (Euclidean distance between the two nodes of the edge.)
- 'tour':  0 (Binary value (0 or 1), indicating whether the edge is part of the optimal tour (1) or not (0).)
Node attribute:
- 'pos':  (0.6049077053425551, 0.5748590937018008) representing the 2D coordinates of the node.


## Model

In [4]:

# --------------------------
# Positional Encoding Class
# --------------------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


# --------------------------
# Encoder Class
# --------------------------
class Encoder(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout, num_layers):
        super(Encoder, self).__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.input_layer = nn.Linear(2, d_model)  # Input: 2D coordinates

    def forward(self, src, src_key_padding_mask=None):
        # Convert input node coordinates to d_model dimensional embeddings
        src = self.input_layer(src)
        return self.encoder(src, src_key_padding_mask=src_key_padding_mask)


# --------------------------
# Decoder Class
# --------------------------
class Decoder(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout, num_layers, num_nodes):
        super(Decoder, self).__init__()
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers)
        self.embedding = nn.Embedding(num_nodes, d_model)
        self.positional_encoding = PositionalEncoding(d_model, dropout)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, num_nodes)
        )

    def forward(self, tgt, memory, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        # Convert node indices to embeddings
        tgt = self.embedding(tgt)

        # Apply positional encoding
        tgt = self.positional_encoding(tgt)

        # Shift the target sequence to prevent cheating
        tgt = torch.cat((tgt.new_zeros((tgt.size(0), 1, tgt.size(2))), tgt[:, :-1, :]), dim=1)

        # Pass through the decoder
        output = self.decoder(
            tgt, memory,
            tgt_key_padding_mask=tgt_key_padding_mask,
            memory_key_padding_mask=memory_key_padding_mask
        )

        # Apply the feedforward network
        return self.ffn(output)


# --------------------------
# TSP Transformer Model
# --------------------------
class TSPTransformer(nn.Module):
    def __init__(self, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout, num_nodes):
        super(TSPTransformer, self).__init__()
        self.encoder = Encoder(d_model, nhead, dim_feedforward, dropout, num_encoder_layers)
        self.decoder = Decoder(d_model, nhead, dim_feedforward, dropout, num_decoder_layers, num_nodes)

    def forward(self, src, tgt, src_key_padding_mask=None, tgt_key_padding_mask=None):
        # Encode the source sequence (node coordinates)
        memory = self.encoder(src, src_key_padding_mask=src_key_padding_mask)

        # Decode the target sequence (node indices)
        output = self.decoder(tgt, memory, tgt_key_padding_mask=tgt_key_padding_mask, memory_key_padding_mask=src_key_padding_mask)

        return output


# --------------------------
# Utility Function to Create Masks
# --------------------------
def create_mask(seq, pad_idx=0):
    """Create a mask for padding tokens."""
    return (seq != pad_idx).unsqueeze(1).unsqueeze(2)


## Training

In [5]:
# --------------------------
# Loss Function and Optimizer
# --------------------------
def get_loss_function():
    return nn.CrossEntropyLoss()

def get_optimizer(model, learning_rate=1e-4):
    optimizer = torch.optim.Adam(model.parameters(), learning_rate, weight_decay=1e-5)
    return optimizer

# --------------------------
# Validation Function
# --------------------------
def validate_epoch(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for src, tgt in tqdm(dataloader, desc="Validation", leave=False):
            src, tgt = src.to(device), tgt.to(device)
            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]
            output = model(src, tgt_input)
            loss = loss_fn(output.reshape(-1, output.shape[-1]), tgt_output.reshape(-1))
            total_loss += loss.item()
    return total_loss / len(dataloader)


### Training WITHOUT gradient accumulation

In [ ]:
# --------------------------
# Standard Training Functions
# --------------------------
def train_epoch(model, dataloader, loss_fn, optimizer, device):
    """
    Train the model for one epoch using standard training.
    """
    model.train()  # Set the model to training mode
    total_loss = 0

    for src, tgt in tqdm(dataloader, desc="Training", leave=False):
        src, tgt = src.to(device), tgt.to(device)

        # Shift the target sequence
        tgt_input = tgt[:, :-1]  # Input sequence (all except the last token)
        tgt_output = tgt[:, 1:]  # Output sequence (all except the first token)

        # Forward pass
        optimizer.zero_grad()  # Clear gradients
        output = model(src, tgt_input)  # Model prediction

        # Calculate loss
        loss = loss_fn(output.reshape(-1, output.shape[-1]), tgt_output.reshape(-1))
        total_loss += loss.item()

        # Backward pass and optimizer step
        loss.backward()
        optimizer.step()

    return total_loss / len(dataloader)


def train_model_standard(model, train_dataloader, val_dataloader, num_epochs, device):

    loss_fn = get_loss_function()
    optimizer = get_optimizer(model)
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")

        # Training for one epoch
        train_loss = train_epoch(model, train_dataloader, loss_fn, optimizer, device)

        # Validation for one epoch
        val_loss = validate_epoch(model, val_dataloader, loss_fn, device)

        # Logging
        print(f"Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

        # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "model_standard.pth")


# --------------------------
# Main Function for Standard Training
# --------------------------
def main_standard_training():
    # Device configuration
    device = torch.device(
        "mps" if torch.backends.mps.is_available() else "cuda:0"
    )

    # Load datasets
    train_dataset = TSPDataset(train_data)
    val_dataset = TSPDataset(val_data)
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Instantiate the model
    model = TSPTransformer(
        d_model=64, nhead=4, num_encoder_layers=3, num_decoder_layers=3,
        dim_feedforward=256, dropout=0.1, num_nodes=20
    ).to(device)

    # Train the model
    num_epochs = 10
    train_model_standard(model, train_dataloader, val_dataloader, num_epochs, device)


# Run the standard training
if __name__ == "__main__":
    main_standard_training()


### Training WITH gradient accumulation

In [ ]:
# --------------------------
# Training with Gradient Accumulation Functions
# --------------------------
def train_epoch_with_accumulation(model, dataloader, loss_fn, optimizer, device, accumulation_steps=4):
    """
    Train the model for one epoch using gradient accumulation.
    """
    model.train()  
    total_loss = 0
    optimizer.zero_grad()  # Clear gradients initially

    for i, (src, tgt) in enumerate(tqdm(dataloader, desc="Training", leave=False)):
        src, tgt = src.to(device), tgt.to(device)

        # Shift the target sequence
        tgt_input = tgt[:, :-1]  # Input sequence (all except the last token)
        tgt_output = tgt[:, 1:]  # Output sequence (all except the first token)

        # Forward pass
        output = model(src, tgt_input)

        # Calculate loss
        loss = loss_fn(output.reshape(-1, output.shape[-1]), tgt_output.reshape(-1))
        total_loss += loss.item()

        # Backward pass (accumulate gradients)
        loss.backward()

        # Perform optimizer step after accumulation_steps batches
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(dataloader):
            optimizer.step()
            optimizer.zero_grad()  # Clearing gradients after step

    return total_loss / len(dataloader)


def train_model_with_accumulation(model, train_dataloader, val_dataloader, num_epochs, device, accumulation_steps=4):

    loss_fn = get_loss_function()
    optimizer = get_optimizer(model)
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")

        # Training for one epoch with gradient accumulation
        train_loss = train_epoch_with_accumulation(
            model, train_dataloader, loss_fn, optimizer, device, accumulation_steps
        )

        # Validation for one epoch
        val_loss = validate_epoch(model, val_dataloader, loss_fn, device)

        # Logging
        print(f"Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

        # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "model_gradient_accumulation.pth")


# --------------------------
# Main Function for Gradient Accumulation Training
# --------------------------
def main_gradient_accumulation_training():
    device = torch.device(
        "mps" if torch.backends.mps.is_available() else "cuda:0"
    )

    # Load datasets
    train_dataset = TSPDataset(train_data)
    val_dataset = TSPDataset(val_data)
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Instantiate the model
    model = TSPTransformer(
        d_model=64, nhead=4, num_encoder_layers=3, num_decoder_layers=3,
        dim_feedforward=256, dropout=0.1, num_nodes=20
    ).to(device)

    # Train the model with gradient accumulation
    num_epochs = 10
    accumulation_steps = 4
    train_model_with_accumulation(model, train_dataloader, val_dataloader, num_epochs, device, accumulation_steps)


# Run the training with gradient accumulation
if __name__ == "__main__":
    main_gradient_accumulation_training()


## Testing

In [ ]:
# --------------------------
# Testing Function
# --------------------------
def test_model(model, test_dataloader, device):
    """
    Test the trained model on the test set and compute the optimality gap.
    """
    model.eval()
    gaps = {'greedy': [], 'random': [], 'transformer_tsp_standard': [], 'transformer_tsp_acc_grad': []}

    with torch.no_grad():
        for graph, _ in tqdm(test_dataloader, desc="Testing", leave=False):
            # Move graph to device
            graph = graph[0]  # Extract the graph from the dataloader
            graph = graph.to(device)

            # Compute gaps for baselines
            gaps['greedy'].append(gap(graph, model=None)['greedy'])
            gaps['random'].append(gap(graph, model=None)['random'])

            # Compute gaps for the model trained with standard training
            model.load_state_dict(torch.load("model_standard.pth"))
            gaps['transformer_tsp_standard'].append(gap(graph, model=model, device=device)['transformer_tsp'])

            # Compute gaps for the model trained with gradient accumulation
            model.load_state_dict(torch.load("model_gradient_accumulation.pth"))
            gaps['transformer_tsp_acc_grad'].append(gap(graph, model=model, device=device)['transformer_tsp'])

    return gaps
# --------------------------
# Plot Gap Distributions
# --------------------------


def plot_gap_distributions(gaps):
    """
    Plot four boxplots showing the gap distributions for random tours,
    the greedy algorithm, and the models (standard training and gradient accumulation).
    """
    plt.figure(figsize=(10, 6))

    # Prepare data for boxplots
    data = [
        gaps['random'],
        gaps['greedy'],
        gaps['transformer_tsp_standard'],
        gaps['transformer_tsp_acc_grad']
    ]
    labels = ['Random Tour', 'Greedy Algorithm', 'Model (Standard)', 'Model (Grad. Accum.)']

    # Create boxplot
    plt.boxplot(data, labels=labels, showmeans=True)
    plt.title("Optimality Gap Distributions")
    plt.ylabel("Gap (%)")
    plt.grid(True)
    plt.show()
# --------------------------
# Main Testing Function
# --------------------------
def main_testing():
    # Device configuration
    device = torch.device("mps" if torch.backends.mps.is_available() else "cuda:0")

    # Load the test dataset
    test_dataset = TSPDataset(test_data)
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)  # One graph at a time

    # Instantiate the model
    model = TSPTransformer(
        d_model=64, nhead=4, num_encoder_layers=3, num_decoder_layers=3,
        dim_feedforward=256, dropout=0.1, num_nodes=20
    ).to(device)

    # Test the model
    gaps = test_model(model, test_dataloader, device)

    # Plot the results
    plot_gap_distributions(gaps)


# Run the testing
if __name__ == "__main__":
    main_testing()